# SIT742: Modern Data Science 
**(Week 03A: Data Wrangling)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.
- If you found any issue/bug for this document, please submit an issue at [tulip-lab/sit742](https://github.com/tulip-lab/sit742/issues)

Prepared by **SIT742 Teaching Team**

---

# Session 03A - DataWrangling with Pandas 

## Table of Content

* Part 1. Scraping data from the web
* Part 2. States and Territories of Australia
* Part 3. Parsing XML files with BeautifulSoup



---

## Part 1. Scraping data from the web

Many of you will probably be interested in scraping data from the web for your projects. For example, what if we were interested in working with some historical Canadian weather data? Well, we can get that from: http://climate.weather.gc.ca using their API. Requests are going to be formatted like this:


In [ ]:
url_template = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=5415&Year={year}&Month={month}&timeframe=1&submit=Download+Data"

Note that we've requested the data be returned as a csv, and that we're going to supply the month and year as inputs when we fire off the query. To get the data for March 2012, we need to format it with month=3, year=2012:


In [ ]:
url = url_template.format(month=3, year=2012)
url

This is great! We can just use the same read_csv function as before, and just give it a URL as a filename. Awesome.

Upon inspection, we find out that there are 16 rows of metadata at the top of this CSV, but pandas knows CSVs are weird, so there's a skiprows options. We parse the dates again, and set 'Date/Time' to be the index column. Here's the resulting dataframe.

In [ ]:
weather_mar2012 = pd.read_csv(url, skiprows=15, index_col='Date/Time', parse_dates=True, encoding='latin1')

In [ ]:
weather_mar2012.head()

As before, we can get rid of any comlumns that don't contain real data using ${\tt .dropna()}$

In [ ]:
weather_mar2012 = weather_mar2012.dropna(axis=1, how='any')

In [ ]:
weather_mar2012.head()

Getting better! The Year/Month/Day/Time columns are redundant, though, and the Data Quality column doesn't look too useful. Let's get rid of those.

In [ ]:
weather_mar2012 = weather_mar2012.drop(['Year', 'Month', 'Day', 'Time'], axis=1)
weather_mar2012[:5]

Great! Now let's figure out how to download the whole year? It would be nice if we could just send that as a single request, but like many APIs this one is limited to prevent people from hogging bandwidth. No problem: we can write a function!

In [ ]:
def download_weather_month(year, month):
    url = url_template.format(year=year, month=month)
    weather_data = pd.read_csv(url, skiprows=15, index_col='Date/Time', parse_dates=True)
    weather_data = weather_data.dropna(axis=1)
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    weather_data = weather_data.drop(['Year', 'Day', 'Month', 'Time'], axis=1)
    return weather_data

Now to test that this function does the right thing:

In [ ]:
download_weather_month(2012, 1).head()

Woohoo! Now we can iteratively request all the months using a single line. This will take a little while to run.

In [ ]:
data_by_month = [download_weather_month(2012, i) for i in range(1, 12)]

Once that's done, it's easy to concatenate all the dataframes together into one big dataframe using ${\tt pandas.concat()}$. And now we have the whole year's data!

In [ ]:
weather_2012 = pd.concat(data_by_month)

This thing is long, so instead of printing out the whole thing, I'm just going to print a quick summary of the ${\tt DataFrame}$ by calling ${\tt .info()}$:

In [ ]:
weather_2012.info()

And a quick reminder, if we wanted to save that data to a file:

In [ ]:
weather_2012.to_csv('weather_2012.csv')

In [ ]:
!ls

And finally, something you should do early on in the wrangling process, plot data:

In [ ]:
# plot that data
import matplotlib.pyplot as plt 
# so now 'plt' means matplotlib.pyplot

In [ ]:
plt.plot(df)

In [ ]:
# nothing to see... in iPython you need to specify where the chart will display, usually it's in a new window
# to see them 'inline' use:
%matplotlib inline

In [ ]:
plt.plot(df)

In [ ]:
# that's better, try other plots, scatter is popular, also boxplot

## Part 2. States and Territories of Australia 

We are interested in getting  State and Territory information from Wikipedia, however we do not want to copy and paste the table : )

Here is the URL
https://en.wikipedia.org/wiki/States_and_territories_of_Australia   

We need two libraries to do the task:

Check documentations here:
* [urllib](https://docs.python.org/2/library/urllib.html)
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)


In [ ]:
import sys
if sys.version_info[0] == 3:
    from urllib.request import urlopen
else:
    from urllib import urlopen
from bs4 import BeautifulSoup

We first save the link in wiki

In [ ]:
wiki = "https://en.wikipedia.org/wiki/States_and_territories_of_Australia"

Then use ulropen to open the page. 

If you get "SSL: CERTIFICATE_VERIFY_FAILED", what you need to do is find where "Install Certificates.command" file is, and click it to upgrade the certificate. Then, you should be able to solve the problem.

In [ ]:
page = urlopen(wiki)

In [ ]:
if sys.version_info[0] == 3:
    page = page.read()

You will meet BeautifulSoup later in this subject, so don't worry if you feel uncomfortable with it now. You can always revisit. 

We begin by reading in the source code and creating a Beautiful Soup object with the BeautifulSoup function.

In [ ]:
soup = BeautifulSoup(page, "lxml")

Then we print and see. 

In [ ]:
 print (soup.prettify())

For who do not know much about HTML, this might be a bit overwhelming, but essentially it contains lots of tags in the angled brackets providing structural and formatting information that we don't care so much here. What we need is the table. 

Let's first check the title.

In [ ]:
soup.title.string

It looks fine, then we would like to find the table. 

Let's have a try to extract all contents within the 'table' tag.

In [ ]:
all_tables = soup.findAll('table')
print(all_tables)

This returns a collection of tag objects. It seems that most of the information are useless and it's getting hard to hunt for the table. So searched online and found an instruction here: 

https://adesquared.wordpress.com/2013/06/16/using-python-beautifulsoup-to-scrape-a-wikipedia-table/

The class is "wikitable sortable"!! Have a try then.   

In [ ]:
right_table=soup.find('table', class_='wikitable sortable')
print (right_table)

Next we need to extract table header row by find the first 'tr'>

In [ ]:
head_row = right_table.find('tr')
print (head_row)

Then we extract header row name via iterate through each row and extract text. 

The .findAll function in Python returns a list containing all the elements, which you can iterate through.

In [ ]:
header_list = []
headers = head_row.findAll('th')
for header in headers:
    #print header.find(text = True)
    header_list.append(header.find(text = True))
header_list

We can probably iterate trough this list and then extract contents. But let's take a simple approach of extracting each column separately. 

In [ ]:
flag=[]
state=[]
abbrev = []
ISO = []
Postal =[]
Type = []
Capital = []
population = []
Area = []
for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells) > 0 and len(cells) == 9:
        flag.append(cells[0].find(text=True))
        state.append(cells[1].find(text=True))
        abbrev.append(cells[2].find(text=True))
        ISO.append(cells[3].find(text=True))
        Postal.append(cells[4].find(text=True))
        Type.append(cells[5].find(text=True))
        Capital.append(cells[6].find(text=True))
        population.append(cells[7].find(text=True))
        Area.append(cells[8].find(text=True))

Next we can append all list to the dataframe.

In [ ]:
df_au = pd.DataFrame()
df_au[header_list[0]] = flag
df_au[header_list[1]] = state
df_au[header_list[2]]=abbrev
df_au[header_list[3]]=ISO
df_au[header_list[4]]=Postal
df_au[header_list[5]]=Type
df_au[header_list[6]]=Capital
df_au[header_list[7]]=population
df_au[header_list[8]]=Area

Done !

In [ ]:
df_au

## Part 3. Parsing XML files with BeautifulSoup

Now, we are going to demonstrate how to use BeautifulSoup to extract information from the XML file, called "Melbourne_bike_share.xml". 

For the documentation of BeautifulSoup, please refer to it <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all">official website</a>. 

In [ ]:
!pip install wget

In [ ]:
import wget

link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Jupyter/data/Melbourne_bike_share.xml'

DataSet = wget.download(link_to_data)

In [ ]:
!ls

In [ ]:
from bs4 import BeautifulSoup
btree = BeautifulSoup(open("Melbourne_bike_share.xml"),"lxml-xml") 

You can alo print out the Beautifulsoup object by calling the <font color="blue">prettify()</font> function.

In [ ]:
print(btree.prettify())

It is easy to figure out information we would like to extract is stored in the following tags
<ul>
<li>id </li>
<li>featurename </li>
<li>terminalname </li>
<li>nbbikes </li>
<li>nbemptydoc </li>
<li>uploaddate </li>
<li>coordinates </li>
</ul>

Each record is stored in "<row> </row>". To extract information from those tags, except for "coordinates", we use the <font color="blue">find_all()</font> function. Its documentation can be found <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all">here</a>.

In [ ]:
featuretags = btree.find_all("featurename")
featuretags

The output shows that the <font color="blue"> find_all() </font> returns all the 50 station names. Now, we need to exclude the tags and just keep the text stored between the tags.

In [ ]:
for feature in featuretags:
    print (feature.string)

Now, we can put all the above code together using list comprehensions. 

In [ ]:
featurenames = [feature.string for feature in btree.find_all("featurename")]

In [ ]:
featurenames

Similarly, we can use the <font color = "blue">find_all()</font> function to extract the other information.

In [ ]:
nbbikes = [feature.string for feature in btree.find_all("nbbikes")]
nbbikes

In [ ]:
NBEmptydoc = [feature.string for feature in btree.find_all("nbemptydoc")]
NBEmptydoc

In [ ]:
TerminalNames = [feature.string for feature in btree.find_all("terminalname")]
TerminalNames

In [ ]:
UploadDate = [feature.string for feature in btree.find_all("uploaddate")]
UploadDate

In [ ]:
ids = [feature.string for feature in btree.find_all("id")]
ids

Now, how can we extract the attribute values from the tage called "coordinates"?

In [ ]:
lattitudes = [coord["latitude"] for coord in btree.find_all("coordinates")]
lattitudes

In [ ]:
longitudes = [coord["longitude"] for coord in btree.find_all("coordinates")]
longitudes

After the extraction, we can put all the information in a Pandas DataFrame.

In [ ]:
import pandas as pd 
dataDict = {}
dataDict['Featurename'] = featurenames
dataDict['TerminalName'] = TerminalNames
dataDict['NBBikes'] = nbbikes
dataDict['NBEmptydoc'] = NBEmptydoc
dataDict['UploadDate'] = UploadDate
dataDict['lat'] = lattitudes
dataDict['lon'] = longitudes
df = pd.DataFrame(dataDict, index = ids)
df.index.name = 'ID'
df.head()